In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_google_repos(pages=1):
    base_url = "https://github.com/orgs/google/repositories?page={}"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0 Safari/537.36"
        )
    }

    results = []

    for page in range(1, pages + 1):
        url = base_url.format(page)
        print(f"\n===== Fetching page {page}: {url} =====")

        # ★ GitHubに負荷をかけない
        time.sleep(1)

        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")

        # GitHubのリポジトリ一覧
        repos = soup.select("li.Box-row")

        for repo in repos:

            # リポジトリ名
            name_tag = repo.select_one('a[data-hovercard-type="repository"]')
            repo_name = name_tag.text.strip() if name_tag else None

            # 主要言語
            lang_tag = repo.select_one('span[itemprop="programmingLanguage"]')
            language = lang_tag.text.strip() if lang_tag else None

            # スター数
            star_tag = repo.select_one('a[href$="/stargazers"]')
            if star_tag:
                star_text = star_tag.text.strip().replace(",", "")
                stars = int(star_text) if star_text.isdigit() else 0
            else:
                stars = 0

            results.append((repo_name, language, stars))

    return results


# ==== 実行して表示 ====
data = scrape_google_repos(pages=1)

print("\n===== スクレイピング結果 =====")
for name, lang, stars in data:
    print(f"Repo: {name} / Lang: {lang} / Stars: {stars}")




===== Fetching page 1: https://github.com/orgs/google/repositories?page=1 =====

===== スクレイピング結果 =====


In [8]:
import sqlite3

def show_data():
    conn = sqlite3.connect("google_repos.db")
    c = conn.cursor()

    c.execute("SELECT id, name, language, stars FROM repositories")
    rows = c.fetchall()

    for row in rows:
        print(row)

    conn.close()


if __name__ == "__main__":
    show_data()
